In [8]:
import pickle
import statsmodels 
import statsmodels.api as sm
from statsmodels.treatment.treatment_effects import TreatmentEffect
import scipy.stats
import pandas as pd
import numpy as np

In [2]:
with open("../../data/acr_ols.pkl", "rb") as f:
    acr_pred_model = pickle.load(f)
    
with open("../../data/logreg.pkl", "rb") as f:
    test_pred_model = pickle.load(f)

In [3]:
def get_diagnoses(df):
    return [d for d in df.columns if d.startswith("d_")]

def get_other_covariates(df):
    return [x for x in df.columns if x.startswith("x_")]

def get_outcome(df):
    return [y for y in df.columns if y.startswith("y_")]


def get_acr_pred_data(df):
    # keep relevant columns
    columns_to_keep = df.filter(regex='^(d|x|y)').columns
    df = df[columns_to_keep]
    df = df.reindex(sorted(df.columns), axis=1)

    # drop y_KREA
    df = df.drop('y_KREA', axis=1)

    # filter out the NaNs
    df.dropna(how='any', axis=0, inplace=True)
    df["x_sex"] = df["x_sex"].map({'M':0, 'F':1})

    # throw out bottom and top 5%
    bottom_percentile = df["y_ACR"].quantile(0.05)
    top_percentile = df["y_ACR"].quantile(0.95)
    df = df.sort_values(by="y_ACR")
    df = df[(df["y_ACR"] > bottom_percentile) & (df["y_ACR"] < top_percentile)]

    y = df["y_ACR"]
    X = sm.add_constant(df.drop('y_ACR', axis=1).astype(float))
    return X, y

In [4]:
real_df = pd.read_csv("../../data/honza_jirka_data.csv")

In [94]:
# len(real_df[~real_df.x_sex.isna()])
test_pred_model.model.exog.shape

(60521, 1479)

In [5]:
X, y = get_acr_pred_data(real_df)
acr_pred_model.model.exog = X
acr_pred_model.model.endog = y

# print(acr_pred_model.model.exog.columns)
# print(test_pred_model.model.exog.columns)



test_pred_model.model.exog = X
#test_pred_model.endog = 1

# Blé

In [98]:
tr = TreatmentEffect(acr_pred_model.model, X["d_I10"], test_pred_model)

In [75]:
X["d_I10"].shape

(9121,)

In [99]:
tr.ipw()

/home/jan/mambaforge/envs/hhh/lib/python3.10/site-packages/statsmodels/treatment/treatment_effects.py:266: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  mm = (w * (endog - treat_ind.dot(tm)))[:, None] * treat_ind


ValueError: operands could not be broadcast together with shapes (60521,) (9126,) 

In [29]:
# def risk(pred_mean, pred_sd, risk_threshold):
#     return 1 - scipy.stats.distributions.norm.cdf(risk_threshold, loc=pred_mean, scale=pred_sd)

# risk(0, 1, -1)

0.8413447460685429

# New

In [ ]:
df = pd.read_csv(Path(__file__).parent.parent / "data" / "honza_jirka_data.csv", delimiter=",")

# keep relevant columns
columns_to_keep = df.filter(regex='^(d|x|y)').columns
df = df[columns_to_keep]
df = df.reindex(sorted(df.columns), axis=1)

# drop y_KREA, map x_sex
df = df.drop('y_KREA', axis=1)
df["x_sex"] = df["x_sex"].map({'M':0, 'F':1})

# get label and drop
y = (~(df["y_ACR"].isna())).astype(int)
df = df.drop("y_ACR", axis=1)

# get indices of rows with valid features
indices = df[df.notna().all(axis=1)].index
y = y.loc[indices]
X = df.loc[indices].astype(float)


# train model on whole data
X = sm.add_constant(X)
logreg_model = sm.Logit(y, X)
result = logreg_model.fit(disp=True, method='lbfgs', maxiter=max_iter)
print(result.summary())

# save model and the columns
#with open('logreg.pkl', 'wb') as f:
#    pickle.dump(result, f)
result.save('logreg.pkl')





# keep relevant columns
columns_to_keep = df.filter(regex='^(d|x|y)').columns
df = df[columns_to_keep]
df = df.reindex(sorted(df.columns), axis=1)

# drop y_KREA
df = df.drop('y_KREA', axis=1)

# filter out the NaNs
df.dropna(how='any', axis=0, inplace=True)
df["x_sex"] = df["x_sex"].map({'M':0, 'F':1})

# throw out bottom and top 5%
bottom_percentile = df["y_ACR"].quantile(0.05)
top_percentile = df["y_ACR"].quantile(0.95)
df = df.sort_values(by="y_ACR")
df = df[(df["y_ACR"] > bottom_percentile) & (df["y_ACR"] < top_percentile)]

y = df["y_ACR"]
X = sm.add_constant(df.drop('y_ACR', axis=1).astype(float))


# p = 0.3
# ind = np.random.choice([True, False], p=(p, 1-p), size=len(X))
ind = np.ones_like(y, dtype=bool)
X_sub = X.loc[ind]
y_sub = y.loc[ind]

mod = sm.GLM(y_sub, X_sub, freq_weights=(1/test_pred_model.predict())[ind])
result = mod.fit()
result.summary()

p = result.get_prediction()

dir(p)
p.summary_frame()


result.save('glm_acr.pkl')

In [54]:
# mod = sm.OLS(y, X)
# result = mod.fit()
# result.summary()

p = 0.3
ind = np.random.choice([True, False], p=(p, 1-p), size=len(X))
# ind = np.ones_like(y, dtype=bool)
X_sub = X.loc[ind]
y_sub = y.loc[ind]

mod = sm.GLM(y_sub, X_sub, freq_weights=(1/test_pred_model.predict())[ind])
result = mod.fit()
result.summary()

p = result.get_prediction()

dir(p)
p.summary_frame()


# glm = smf.glm(
#     "affairs ~ rate_marriage + age + yrs_married",
#     data=dc,
#     family=sm.families.Poisson(),
#     freq_weights=np.asarray(dc["freq"]),
# )
# res_f = glm.fit()
# print(res_f.summary())

,mean,mean_se,mean_ci_lower,mean_ci_upper
23611,-0.064059,0.168761,-0.394823,0.266706
44913,2.090701,0.030705,2.030520,2.150882
31115,0.029706,0.333177,-0.623309,0.682721
44774,7.048457,0.222006,6.613333,7.483581
36685,2.026672,0.037503,1.953168,2.100175
...,...,...,...,...
40344,57.180611,0.034342,57.113301,57.247921
9036,51.776801,0.321808,51.146069,52.407533
31038,43.101822,0.229421,42.652166,43.551478
49806,12.442539,0.280322,11.893119,12.991959


In [55]:
p.summary_frame().describe()

,mean,mean_se,mean_ci_lower,mean_ci_upper
count,2722.000000,2722.000000,2722.000000,2722.000000
mean,5.513714,0.290204,4.944925,6.082502
std,8.215886,0.217894,8.220306,8.233644
min,-20.252072,0.001457,-21.088860,-19.415284
25%,1.351226,0.097178,0.783055,1.948644
50%,3.658123,0.242030,3.243063,4.077519
75%,7.263421,0.438987,6.803255,7.748551
max,58.305798,0.808882,57.113301,59.822027


In [41]:
len(test_pred_model.predict()), len(X), len(X_sub)

(9126, 9126, 903)